## Import Library

In [1]:
from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import numpy as np
import imutils
import cv2

## Def middlePoint

In [2]:
def midpoint(ptA,ptB):
    return ((ptA[0]+ptB[0])*0.5,(ptA[1]+ptB[1])*0.5)

## Load YOLO

In [3]:
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

## Load Image

In [4]:
image = cv2.imread("./img/sample07.jpg")
image = cv2.resize(image, None, fx=3, fy=3)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (7, 7), 0)
height, width, channels = image.shape

## Searching Object

In [5]:
blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

In [6]:
class_ids = []
confidences = []
boxes = []
boxes2 = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            # 좌표
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([[x, y], [x+w,y],[x,y+h],[x+w,y+h]])
            boxes2.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

In [7]:
print(len(boxes))

9


In [8]:
boxes

[[[82, 183], [263, 183], [82, 322], [263, 322]],
 [[259, 110], [526, 110], [259, 418], [526, 418]],
 [[298, 113], [549, 113], [298, 412], [549, 412]],
 [[291, 138], [554, 138], [291, 437], [554, 437]],
 [[106, 161], [251, 161], [106, 316], [251, 316]],
 [[94, 174], [236, 174], [94, 329], [236, 329]],
 [[105, 174], [253, 174], [105, 330], [253, 330]],
 [[105, 186], [250, 186], [105, 338], [250, 338]],
 [[55, 310], [231, 310], [55, 337], [231, 337]]]

## Find MiddlePoint

In [9]:
middlePoints = []
for box in boxes:
    box = np.array(box)
    box = perspective.order_points(box)
    print(box)
    (tl, tr, br, bl) = box
    (tltrX, tltrY) = midpoint(tl, tr)
    (blbrX, blbrY) = midpoint(bl, br)
    (tlblX, tlblY) = midpoint(tl, bl)
    (trbrX, trbrY) = midpoint(tr, br)
    middlePoints.append([(tltrX, tltrY),(blbrX, blbrY),(tlblX, tlblY),(trbrX, trbrY)])

[[ 82. 183.]
 [263. 183.]
 [263. 322.]
 [ 82. 322.]]
[[259. 110.]
 [526. 110.]
 [526. 418.]
 [259. 418.]]
[[298. 113.]
 [549. 113.]
 [549. 412.]
 [298. 412.]]
[[291. 138.]
 [554. 138.]
 [554. 437.]
 [291. 437.]]
[[106. 161.]
 [251. 161.]
 [251. 316.]
 [106. 316.]]
[[ 94. 174.]
 [236. 174.]
 [236. 329.]
 [ 94. 329.]]
[[105. 174.]
 [253. 174.]
 [253. 330.]
 [105. 330.]]
[[105. 186.]
 [250. 186.]
 [250. 338.]
 [105. 338.]]
[[ 55. 310.]
 [231. 310.]
 [231. 337.]
 [ 55. 337.]]


## Estimating Size and Visualize

In [10]:
indexes = cv2.dnn.NMSBoxes(boxes2, confidences, 0.5, 0.4)

In [ ]:
orig = image.copy()
font = cv2.FONT_HERSHEY_PLAIN
pixelsPerMetric = None
colors = np.random.uniform(0, 255, size=(len(classes), 3))
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes2[i]
        color = colors[i]
        label = str(classes[class_ids[i]])+" "+str(confidences[i])
        cv2.rectangle(orig, boxes[i][0], boxes[i][3], color, 2)
        cv2.putText(orig, label, (x, y + 30), font, 1, color, 2)
        for (x, y) in boxes[i]:
            cv2.circle(orig, (int(x), int(y)), 5, (0, 0, 255), -1)
        for (x, y) in middlePoints[i]:
            cv2.circle(orig, (int(x), int(y)), 5, (255, 0, 0), -1)
    #     cv2.line(orig, middlePoints[i][0], middlePoints[i][1],(255, 0, 255), 2)
    #     cv2.line(orig, middlePoints[i][2], middlePoints[i][3],(255, 0, 255), 2)
        dA = dist.euclidean(middlePoints[i][0], middlePoints[i][1])
        dB = dist.euclidean(middlePoints[i][2], middlePoints[i][3])
        print(dA,dB)
        if pixelsPerMetric is None:
            pixelsPerMetric = dB / 3
        dimA = dA / pixelsPerMetric
        dimB = dB / pixelsPerMetric
        cv2.putText(orig, "{:.1f}in".format(dimB),(int(middlePoints[i][0][0] - 15), int(middlePoints[i][0][1] - 10)), cv2.FONT_HERSHEY_SIMPLEX,0.65, (0, 0, 0), 2)
        cv2.putText(orig, "{:.1f}in".format(dimA),(int(middlePoints[i][3][0] + 10), int(middlePoints[i][3][1])), cv2.FONT_HERSHEY_SIMPLEX,0.65, (0, 0, 0), 2)
cv2.imshow("Image", orig)
cv2.waitKey(0)
cv2.destroyAllWindows()

299.0 251.0
156.0 148.0
27.0 176.0
